In [99]:
from unidecode import unidecode
import string as string
import geonamescache
import re
from pandas import DataFrame

def getCities():
    geonames = geonamescache.GeonamesCache()
    normalizedNames = {}
    for city in geonames.get_cities().values():
        cityName = unidecode(city["name"]).lower()
        cityName = cityName.translate(str.maketrans('', '', string.punctuation))
        normalizedNames[cityName] = {"name": city["name"] , "country": city["countrycode"]}
    return normalizedNames


In [95]:
class Headline:
    CITY_NAMES = getCities()
    def __init__(self,text):
        self.text = text.replace("\n","")
        self.city = None
        self.country = None
    def _setCity(self,city):
        self.country = CITY_NAMES[city]["country"]
        self.city = CITY_NAMES[city]["name"]
    def findCity(self, name):
        if name in CITY_NAMES:
            city = name
            if not self.city:
                self._setCity(city)
            elif len(CITY_NAMES[city]["name"]) > len(self.city):
                self._setCity(city)
    def __str__(self):
        return f'{{"headline": "{self.text}", "city": "{self.city}", "country": "{self.country}"}}'
    def to_dict(self):
        return {"headline":self.text, "city":self.city, "country":self.country}



In [96]:
def HeadlineIterator():
    for headline in open('headlines.txt', 'r'):
        yield headline



In [97]:
def FindLocation(inHeadline):
    word_list = re.split(r'\b',inHeadline)
    word_list = [word.translate(str.maketrans('', '', string.punctuation)) for word in word_list]
    word_list = [word.lower() for word in word_list]
    word_list_length = len(word_list)
    candidate = ''
    headline = Headline(inHeadline)
    for i in range(0,word_list_length+1):
        for j in range(i+1,word_list_length+1):
            candidate = "".join(word_list[i:j])
            headline.findCity(candidate)
    return headline



In [98]:
parsed = {"headline": [], "city": [], "country": []}
for headline in HeadlineIterator():
    headline = FindLocation(headline)
    if headline.city:
        parsed["headline"].append(headline.text)
        parsed["city"].append(headline.city)
        parsed["country"].append(headline.country)

dataFrame = DataFrame(data=parsed)

print(dataFrame)

                                              headline           city country
0                             Zika Outbreak Hits Miami          Miami      US
1                      Could Zika Reach New York City?  New York City      US
2                    First Case of Zika in Miami Beach    Miami Beach      US
3              Mystery Virus Spreads in Recife, Brazil         Recife      BR
4              Dallas man comes down with case of Zika         Dallas      US
..                                                 ...            ...     ...
615  Rumors about Rabies spreading in Jerusalem hav...      Jerusalem      IL
616              More Zika patients reported in Indang         Indang      PH
617  Suva authorities confirmed the spread of Rotav...           Suva      FJ
618         More Zika patients reported in Bella Vista    Bella Vista      US
619                     Zika Outbreak in Wichita Falls  Wichita Falls      US

[620 rows x 3 columns]
